<a href="https://colab.research.google.com/github/Linaqruf/Scraper/blob/main/Proseka_Wiki_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import json
import os
import threading
from bs4 import BeautifulSoup
from pprint import pprint

def download(directory, url):
    if not os.path.exists(directory + url.split('/')[-1]):
        with open(directory + url.split('/')[-1], 'wb') as f:
            f.write(requests.get(url).content)
    else:
        return 0

def get_rarity(img_url):
    if 'Ribbon' in img_url:
        return '🎀'
    elif 'Star_4_trained' in img_url:
        return '⭐⭐⭐⭐'
    elif 'Star_3_trained' in img_url:
        return '⭐⭐⭐'
    elif 'Star_2_untrained' in img_url:
        return '⭐⭐'
    elif 'Star_untrained' in img_url:
        return '⭐'
    else:
        return ''

urls = [
    "https://projectsekai.fandom.com/wiki/Cards_1-50", \
    "https://projectsekai.fandom.com/wiki/Cards_51-100", \
    "https://projectsekai.fandom.com/wiki/Cards_101-150", \
    "https://projectsekai.fandom.com/wiki/Cards_201-250", \
    "https://projectsekai.fandom.com/wiki/Cards_251-300", \
    "https://projectsekai.fandom.com/wiki/Cards_301-350", \
    "https://projectsekai.fandom.com/wiki/Cards_351-400", \
    "https://projectsekai.fandom.com/wiki/Cards_401-450", \
    "https://projectsekai.fandom.com/wiki/Cards_451-500", \
    "https://projectsekai.fandom.com/wiki/Cards_501-550", \
    "https://projectsekai.fandom.com/wiki/Cards_551-600"
]

for url in urls:
  final = []
  basename = os.path.basename(url) 
  for i in BeautifulSoup(requests.get(url).content, 'lxml').find('table').findAll('tr')[1:]:
      td_elements = i.findAll('td')
      data = []
      for j in td_elements:
          if '<br/>' in str(j):
              names = []
              for link in j.findAll("a"):
                  names.append(link.text.strip())
              data.append(": ".join(names))
          else:
              data.append(j.text.strip())
      rarity = get_rarity(re.findall(r'src="(https:\/\/.*?)"', str(td_elements[0].find('img')))[0])
      id = int(data[1])
      name = data[2]
      untrained_url = None
      trained_url = None
      if td_elements[3].find('img'):
        untrained_url = '/'.join(re.findall(r'src="(https:\/\/.*?)"', str(td_elements[3].find('img')))[0].split('/')[:8])
      if td_elements[4].find('img'):
        trained_url = '/'.join(re.findall(r'src="(https:\/\/.*?)"', str(td_elements[4].find('img')))[0].split('/')[:8])
      final.append({"rarity": rarity, "id": id, "name": name, "untrained_url": untrained_url, "trained_url": trained_url})

  os.makedirs(f'proseka/untrained_url/{basename}', exist_ok=True)
  os.makedirs(f'proseka/trained_url/{basename}', exist_ok=True)
  os.makedirs(f'proseka/metadata', exist_ok=True)

  for i in (i['untrained_url'] for i in final):
      threading.Thread(target=download, args=(f'proseka/untrained_url/{basename}/', i)).start()

  for i in (i['trained_url'] for i in final if i['trained_url']):
      threading.Thread(target=download, args=(f'proseka/trained_url/{basename}/', i)).start()
      
  with open(f'proseka/metadata/{basename}.json', 'w', encoding='utf-8') as f:
      json.dump(final, f, indent=4, ensure_ascii=False)
